In [4]:
import pandas as pd
import os
import time
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from model import LineageMantaObjectOptimize
from utils import create_raw_table, transfrom_column_name_before_create_table, convert_df_to_oracle_format
from config import NODE_PATH, LINEAGE_SOURCE, LINEAGE_TARGET,EXPORT_MANTA_FILE_PATH, LINEAGE_MANTA_RAW_TEST,LINEAGE_MANTA_RAW_MAIN,LINEAGE_MANTA_FLOW_TEST, LINEAGE_MANTA_FLOW_MAIN,  TABLE_MANTA_LINEAGE_RAW_COLUMN, IMPORT_MANTA_FILE_PATH, TABLE_MANTA_LINEAGE_FLOW
from repo import RepoLineage

In [5]:
if __name__ == '__main__':
    # start_time = time.time()
    object = LineageMantaObjectOptimize(lineage_name= 'manta')
    df_raw, table_name_list = create_raw_table(object)
    df_graph = None
# df_raw.to_csv(f'{EXPORT_MANTA_FILE_PATH}/df_raw.csv', index = False)
   

LINEAGE_MANTA_W4
TABLE_NAME =  FM_CLIENT_CONTACT | FILE_NAME=  relations_FM_CLIENT_CONTACT | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_FM_CLIENT_CONTACT.csv
TABLE_NAME =  FM_CLIENT_INDVL | FILE_NAME=  relations_FM_CLIENT_INDVL | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_FM_CLIENT_INDVL.csv
TABLE_NAME =  HDB | FILE_NAME=  relations_HDB | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_HDB.csv
TABLE_NAME =  RB_ACCT | FILE_NAME=  relations_RB_ACCT | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_RB_ACCT.csv
TABLE_NAME =  RB_TRAN_HIST | FILE_NAME=  relations_RB_TRAN_HIST | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_RB_TRAN_HIST.csv


In [6]:
print(df_raw.columns)
# print(df_raw['UPDATE_TIME'].head(5))

Index(['Type', 'SourcePath', 'TargetPath', 'SourceColumnName',
       'SourceColumnType', 'TargetColumnName', 'TargetColumnType',
       'SourceObjectName', 'SourceObjectType', 'TargetObjectName',
       'TargetObjectType', 'SourceGroupName', 'SourceGroupType',
       'TargetGroupName', 'TargetGroupType', 'SourceResourceName',
       'SourceResourceType', 'TargetResourceName', 'TargetResourceType',
       'RevisionState', 'Table_name', 'UPDATE_TIME'],
      dtype='object')


In [7]:
for table_name in table_name_list:
    # time.sleep(5)
    df = df_raw.query(f"Table_name == '{table_name}'")
    if len(df) >= 1:
        df_describe_op = object.describe_table_optimized(
            df_path=df,
            source_col=LINEAGE_SOURCE,
            target_col=LINEAGE_TARGET,
        )
        all_values = list(df_describe_op.get("all_values"))
        redact_node_df = object.redact_node_with_int(total_nodes_df=df.copy())
        combo_list = redact_node_df[NODE_PATH].tolist()
        # combo_list = [1,2,1868,72,824,928,40,121,366,1691,35,548,663
        # ,1944,509,638,1800,620,750,54,1509,2166,1531,1173,1013,74,345,245
        # ,762,2165,1439,1525,75,631,840,1734,1738,2001,728,1728,1987,1341,145,1724]
        dictionary_op = object.mapping_dict_nodepath_optimize(df =redact_node_df)
        adjacency_list = object.create_adjacency_list_optimized(dictionary = dictionary_op)
        graph = object.deploy_lineage_manta_optimized(df_raw= df.copy(), 
                                    combo_list_copy=combo_list, 
                                    dictionary=dictionary_op,
                                    adjacency_list = adjacency_list, 
                                    source_col=LINEAGE_SOURCE, 
                                    target_col=LINEAGE_TARGET)
        object.add_name_of_table_column(df = graph, table_name = table_name)
        if df_graph is None:
            df_graph = graph
        else:
            df_graph = pd.concat([df_graph, graph])
    else:
        print(f"Table {table_name} is empty")
pd.set_option('display.max_rows', None)

Table HDB is empty


In [5]:
# object.export_to_csv(df=df_graph, file_path=f'{EXPORT_MANTA_FILE_PATH}/df_graph.csv', index = False)

In [ ]:
# df_graph = object.read_data_from_file(f'{IMPORT_MANTA_FILE_PATH}/df_graph.csv')

In [8]:
df_graph = object.extract_components_optimize(table = df_graph, column_extract='Raw_Node')
df_graph = transfrom_column_name_before_create_table(df=df_graph)

In [9]:
print(df_graph.head(100))
print(len(df_graph))

        

                                             FLOW_RAW  \
0   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
1   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
2   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
3   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
4   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
5   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
6   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
7   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
8   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
9   Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
10  Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
11  Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
12  Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
13  Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
14  Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
15  Oracle.EOCN4.EOC.CL_FEES.FEE_POSTED,Oracle.EOC...   
16  Oracle.EOCN4.EOC.CL_FEES.FE

In [10]:
database = 'test'
enviroment = 'DEV'
db_connection = RepoLineage()
db_connect = db_connection.get_connection(db_name='test',environment = 'DEV', user = 'dtc')

Connection to database 'test' established successfully.


In [ ]:
# data = db_connection.create_lineage_manta_table(connection = db_connect,
#                                                 table_name = LINEAGE_MANTA_RAW_MAIN,
#                                                 column_definitions= TABLE_MANTA_LINEAGE_RAW_COLUMN)
# db_connection.insert_data_from_dataframe(connection = db_connect, table_name = LINEAGE_MANTA_RAW_MAIN, dataframe=convert_df_to_oracle_format(transfrom_column_name_before_create_table(df_raw)))

In [ ]:
# df_raw.to_csv(f'{EXPORT_MANTA_FILE_PATH}/df_raw.csv', index = False)# 

In [11]:
data = db_connection.create_lineage_manta_table(connection = db_connect,
                                                table_name = LINEAGE_MANTA_FLOW_MAIN,
                                                column_definitions= TABLE_MANTA_LINEAGE_FLOW)
db_connection.insert_data_from_dataframe(connection = db_connect, table_name = LINEAGE_MANTA_FLOW_MAIN, dataframe=convert_df_to_oracle_format(transfrom_column_name_before_create_table(df_graph)))

In [12]:
db_connection.rename_column(connection = db_connect, table_name = LINEAGE_MANTA_FLOW_MAIN, old_column_name = 'COLUMN_NAME', new_column_name = 'COLUMN_NAME_OBJECT')